## Парсинг статей Хабр JSON

### 1.1 Импорт модулей

In [3]:
# Для работы с данными
import pandas as pd
import numpy as np

# Для работы с json
import json

# Для получения пути до всех json файлов
import glob

# Для парсинга даты
import dateparser

### 1.2 Парсинг данных

Сначала необходимо получить пути до всех json файлов, для этого будем использовать библиотеку glob, которая удобно запишит все пути в список

In [4]:
all_jsons = glob.glob(r"jsons\*.json")
all_jsons

['jsons\\Cybersport.Metaratings.json',
 'jsons\\ESforce Holding.json',
 'jsons\\Gett.json',
 'jsons\\Goose Gaming.json',
 'jsons\\MMA.Metaratings.json',
 'jsons\\skillbox.json',
 'jsons\\Vinci Agency.json',
 'jsons\\Гэллэри Сервис.json',
 'jsons\\Здоровье.ру.json',
 'jsons\\Иннотех.json',
 'jsons\\Проект по использованию технологий компьютерного зрения на базе искусственного интеллекта для анализа медицинских изображений.json',
 'jsons\\Промобот.json',
 'jsons\\Студия Артемия Лебедева.json',
 'jsons\\Федерация креативных индустрий.json']

Хорошо, раз теперь все пути имеются, можно приступать к парсингу. Стоит отметить, что каждый json файл имеет следующую структуру: имеется один основной словарь, который хранит данные о компании, которая выкладывает статьи. 
Такой словарь содержит информацию о компании, статьи, написанные компанией и количество написанных статей.  
Информация о компании может содержать словарь с: рейтингом компании, подписчиками, информацию о компании и направления в которых она работает.  
Статьи содержат в себе: текст статьи, словарь с датой и словарь с рейтингом и просмотрами статьи.  

Теперь, когда структура известна, можно приступать к парсингу.

In [5]:
# Определяем словарь с будущими столбцами
dict_articles = {'company':[], 'company_rate':[], 'subs':[], 'industries':[], 'about_company':[], 'text':[], 'publish_date':[], 'rate':[], 'views':[], 'refs_quant':[]}

# В основном цикле будем перебирать все компании
for json_file in all_jsons:
    # Читаме json файл
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Записываем название компании
    company = json_file.split('\\')[-1].split('.json')[0]
    # Записываем количество написанных статей
    refs_quant = data['refs_quant']
    
    #print(f'---------------{company}---------------')

    # Вытаскиваем основную информацию о компании
    info = data['info']

    # Определяем рейтинг компании, кол-во подписчиков, направления в котором работает компания, описание о компании, как нулевые значения
    company_rate = np.nan
    subs = np.nan
    industries = np.nan
    about_company = np.nan

    # Если же информация присутствует, то записываем информацию о компании
    if info is not None:
        company_rate = info['rate']
        subs = info['subs']
        industries = ', '.join(info['industries'])
        about_company = info['about']
        
    '''
    print(f"""
          Рейтинг компании - {company_rate}
          Кол-во подписчиков - {subs}
          Направления - {industries}
          О компании - {about_company}
          """)
    '''

    # Теперь в цикле перебираем все статьи, которая написала компания
    index = 1
    for article in data['refs']:
        # Если статья имеет неверный формат и скорее всего null, то пропускаем её
        if not isinstance(article, list):
            continue

        # Иначе записываем текст статьи
        text = article[0]
        datetime_dict = article[1]
        article_info = article[2]

        # Записываем дату публикации статьи, парся её через модуль dateparser
        datetime_str = datetime_dict['day'] + datetime_dict['month'] + datetime_dict['time']
        publish_date = dateparser.parse(datetime_str, languages=['ru'])

        # Записываем кол-во оценок и просмотров на статье
        rate = article_info['rate']
        views = article_info['views']

        '''
        print(f"""
              {index}
              Дата публикации - {publish_date}
              Оценок - {rate}
              Просмотров - {views}
              """)
        index += 1
        '''

        # Записываем все получившееся данные в словарь
        dict_articles['company'].append(company)
        dict_articles['company_rate'].append(company_rate)
        dict_articles['refs_quant'].append(refs_quant)
        dict_articles['subs'].append(subs)
        dict_articles['industries'].append(industries)
        dict_articles['about_company'].append(about_company)
        dict_articles['text'].append(text)
        dict_articles['publish_date'].append(publish_date)
        dict_articles['rate'].append(rate)
        dict_articles['views'].append(views)

Данные о статьях успешно записаны в словарь, а значит можно формировать итоговый датасет.

In [6]:
df = pd.DataFrame(dict_articles)
df

,company,company_rate,subs,industries,about_company,text,publish_date,rate,views,refs_quant
0,Cybersport.Metaratings,NaN,NaN,NaN,NaN,7 декабря состоялась церемония награждения лау...,2025-12-08 09:30:00,6,2300,1
1,ESforce Holding,NaN,NaN,NaN,NaN,7 декабря состоялась церемония награждения лау...,2025-12-08 09:30:00,6,2300,4
2,ESforce Holding,NaN,NaN,NaN,NaN,Из совместного исследования издателя компьютер...,2025-07-29 13:50:00,12,2800,4
3,ESforce Holding,NaN,NaN,NaN,NaN,По статистике The Esports Observer за I полови...,2025-07-09 09:33:00,18,6200,4
4,ESforce Holding,NaN,NaN,NaN,NaN,Компании SuperData Research и PayPal провели с...,2025-04-11 12:43:00,11,10000,4
...,...,...,...,...,...,...,...,...,...,...
1107,Федерация креативных индустрий,NaN,NaN,NaN,NaN,"Привет, Хабр! Представляю вашему вниманию пере...",2025-01-30 10:43:00,32,9300,10
1108,Федерация креативных индустрий,NaN,NaN,NaN,NaN,Источник\n«Умный человек не делает все ошибки ...,2025-04-24 12:37:00,3,7700,10
1109,Федерация креативных индустрий,NaN,NaN,NaN,NaN,"Привет, Хабр! Сегодня мы решили поделиться с с...",2025-10-12 14:09:00,19,9300,10
1110,Федерация креативных индустрий,NaN,NaN,NaN,NaN,"Привет! Меня зовут Никита Бокарев, я креативны...",2025-09-04 16:44:00,52,52000,10


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   company        1112 non-null   object        
 1   company_rate   293 non-null    float64       
 2   subs           293 non-null    float64       
 3   industries     293 non-null    object        
 4   about_company  293 non-null    object        
 5   text           1112 non-null   object        
 6   publish_date   1111 non-null   datetime64[ns]
 7   rate           1112 non-null   int64         
 8   views          1112 non-null   int64         
 9   refs_quant     1112 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 87.0+ KB


In [8]:
df.to_csv("dataset_json.csv", index=False)